CatBoost를 사용한 모델 학습 / Hyperopt를 사용한 파라미터 튜닝 / 테스트데이터 호출 및 결과 데이터 저장까지 구현되어 있습니다.
코드가 너무 많아 모든 코드에 자세한 주석을 달 지 못하여 죄송합니다.

In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from hyperopt import STATUS_OK


In [2]:
data = pd.read_csv("train_landmark.csv")

In [3]:
X = data.values[:,:-1]
y = data.values[:,-1]

In [4]:
row_means = np.mean(X, axis=1)
X_centerized = X - row_means.reshape(-1, 1)
label = LabelEncoder()
int_y = label.fit_transform(y)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, int_y, test_size=0.2)

In [39]:
def objective(search_space):
    model = CatBoostClassifier(**search_space,
                              early_stopping_rounds=100,
                              random_seed=42)
    model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)
    return {'loss': model.get_best_score()['validation']['RMSE'], 'status': STATUS_OK}

In [ ]:
# Learning space의 설정. Hyperopt가 탐색할 parameter의 범위를 결정합니다.
learning_rate=np.linspace(0.01,0.1,10)
max_depth=np.arange(2, 18, 2)
colsample_bylevel=np.arange(0.3, 0.8, 0.1)
iterations=np.arange(50, 1000, 50)
l2_leaf_reg=np.arange(0,10)
bagging_temperature=np.arange(0,100,10)
space = {
    'learning_rate': hp.choice('learning_rate', learning_rate),
    'max_depth': hp.choice('max_depth', max_depth),
    'colsample_bylevel': hp.choice('colsample_bylevel', colsample_bylevel),
    'iterations': hp.choice('iterations', iterations),
    'l2_leaf_reg': hp.choice('l2_leaf_reg', l2_leaf_reg),
    'bagging_temperature':hp.choice('bagging_temperature', bagging_temperature),
}

# 최소화할 objective function을 설정하고, accuracy의 음수값으로 설정합니다.
def objective(params):
    clf = CatBoostClassifier(
        iterations=params['iterations'],
        random_seed=42,
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        colsample_bylevel=params['colsample_bylevel'],
        l2_leaf_reg=params['l2_leaf_reg'],
        bagging_temperature=params['bagging_temperature'],
        loss_function='MultiClass'
    )
    clf.fit(X_train, y_train, verbose=False, early_stopping_rounds=50)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy

# Optimization을 싫행합니다.
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=5, trials=trials)

In [76]:
best # 각각 주어진 값들의 index에 해당합니다.

{'bagging_temperature': 0,
 'colsample_bylevel': 0,
 'iterations': 13,
 'l2_leaf_reg': 8,
 'learning_rate': 8,
 'max_depth': 5}

In [80]:
clf = CatBoostClassifier(bagging_temperature=bagging_temperature[0],
        colsample_bylevel = colsample_bylevel[0],
        iterations=iterations[13],
        l2_leaf_reg=l2_leaf_reg[8],
        learning_rate=learning_rate[8],
        max_depth=max_depth[5]
)
clf.fit(X_train, y_train, early_stopping=50)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

0:	learn: 1.6985055	total: 860ms	remaining: 10m 1s
1:	learn: 1.6039317	total: 1.35s	remaining: 7m 53s
2:	learn: 1.5056888	total: 1.88s	remaining: 7m 17s
3:	learn: 1.4320745	total: 2.46s	remaining: 7m 7s
4:	learn: 1.3659570	total: 2.95s	remaining: 6m 49s
5:	learn: 1.3053379	total: 3.48s	remaining: 6m 41s
6:	learn: 1.2455793	total: 3.98s	remaining: 6m 33s
7:	learn: 1.1990874	total: 4.48s	remaining: 6m 27s
8:	learn: 1.1573337	total: 4.98s	remaining: 6m 22s
9:	learn: 1.1213553	total: 5.49s	remaining: 6m 18s
10:	learn: 1.0789676	total: 6.07s	remaining: 6m 20s
11:	learn: 1.0420222	total: 6.57s	remaining: 6m 16s
12:	learn: 1.0072724	total: 7.09s	remaining: 6m 14s
13:	learn: 0.9718526	total: 7.61s	remaining: 6m 13s
14:	learn: 0.9436320	total: 8.19s	remaining: 6m 13s
15:	learn: 0.9168339	total: 8.66s	remaining: 6m 10s
16:	learn: 0.8915548	total: 9.08s	remaining: 6m 4s
17:	learn: 0.8721297	total: 9.51s	remaining: 6m
18:	learn: 0.8483389	total: 10s	remaining: 5m 58s
19:	learn: 0.8282540	total: 10

In [81]:
clf.fit(X, int_y)

0:	learn: 1.6763086	total: 839ms	remaining: 9m 46s
1:	learn: 1.5745364	total: 1.5s	remaining: 8m 45s
2:	learn: 1.4675203	total: 1.98s	remaining: 7m 39s
3:	learn: 1.3973917	total: 2.53s	remaining: 7m 20s
4:	learn: 1.3294859	total: 2.99s	remaining: 6m 55s
5:	learn: 1.2544883	total: 3.51s	remaining: 6m 45s
6:	learn: 1.1966112	total: 3.98s	remaining: 6m 34s
7:	learn: 1.1465196	total: 4.5s	remaining: 6m 29s
8:	learn: 1.1029507	total: 4.99s	remaining: 6m 23s
9:	learn: 1.0669323	total: 5.5s	remaining: 6m 19s
10:	learn: 1.0162186	total: 6.1s	remaining: 6m 22s
11:	learn: 0.9752024	total: 6.62s	remaining: 6m 19s
12:	learn: 0.9427937	total: 7.16s	remaining: 6m 18s
13:	learn: 0.9137986	total: 7.69s	remaining: 6m 17s
14:	learn: 0.8842379	total: 8.3s	remaining: 6m 19s
15:	learn: 0.8580518	total: 8.81s	remaining: 6m 16s
16:	learn: 0.8322446	total: 9.25s	remaining: 6m 11s
17:	learn: 0.8124913	total: 9.7s	remaining: 6m 7s
18:	learn: 0.7901741	total: 10.2s	remaining: 6m 5s
19:	learn: 0.7691305	total: 10

In [82]:
# 테스트 데이터 호출
test = pd.read_csv("test_landmark.csv")
test_X = test.iloc[:,:-1]
nan_rows = test_X.isnull().any(axis=1)
nan_idx = nan_rows[nan_rows].index
nan_idx

# 무작위로 다른 행의 값으로 대체
test_X.loc[230] = test_X.loc[np.random.choice(test.index)]
test_X.loc[345] = test_X.loc[np.random.choice(test.index)]

In [90]:
# 결과물이 np array를 요소로 갖는 리스트로 반환됩니다. 
results = np.concatenate(clf.predict(test_X))
Id = [f'{i}.jpg' for i in range(0, 389)]

In [91]:
result = pd.DataFrame({'Id':Id, 'Category':results})

In [92]:
result.to_csv("predict_catboost.csv", index=False)